In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
pd.options.mode.chained_assignment = None

Собираем данные о демографии некоторых стран с Википедии

In [2]:
urls = ['https://en.wikipedia.org/wiki/Demographics_of_Australia',
       'https://en.wikipedia.org/wiki/Demographics_of_Malaysia',
       'https://en.wikipedia.org/wiki/Demographics_of_Sweden',
       'https://en.wikipedia.org/wiki/Demographics_of_Switzerland',
       'https://en.wikipedia.org/wiki/Demographics_of_Belgium']
countries = ['Australia', 'Malaysia', 'Sweden', 'Switzerland', 'Belgium']

In [ ]:
result_df = pd.DataFrame()

In [ ]:
for i, url in enumerate(urls):
    dfs = pd.read_html(url)
    current_df = None

    for df in dfs:
        if 'Average population' in df.columns: #выбираем нужную табличку
            current_df = df
            break

    #Без имени может быть только колонка - год, и в целом все таблицы имеют одинаковые названия колонок
    #так что можем применять универсальные модификации
    current_df.rename(columns = {"Unnamed: 0" : 'Year'}, inplace = True)
    #в результат пойдут эти 5 колонок
    current_df = current_df[["Year", "Average population", "Live births", "Deaths", "Natural change"]]

    for column in current_df.columns:
        if current_df.dtypes[column] not in ['int64', 'float64', 'float']:
            #если в ячейке была ссылка вида [xx] - отрезаем эту ссылку
            current_df[column] = current_df[column].map(lambda x: x[:4] if x.find('[') != -1 else x)
            #убираем лишние символы, оставляем только цифры и возможные модификации минуса.
            #у нас нет дробных чисел, так что точки и запятые тоже можно убрать
            current_df[column] = current_df[column].map(lambda x: re.sub("[^0-9 - — –]", "", x))
            current_df[column] = current_df[column].astype("int32")

    current_df['Country'] = countries[i]

    result_df = pd.concat([result_df, current_df], ignore_index=True)


In [ ]:
result_df

,Year,Average population,Live births,Deaths,Natural change,Country
0,1900,3715000,102221.0,44060.0,27.3,Australia
1,1901,3765000,102945.0,46330.0,56615.0,Australia
2,1902,3824000,102776.0,48078.0,54698.0,Australia
3,1903,3875000,98443.0,47293.0,51150.0,Australia
4,1904,3916000,104113.0,43572.0,60541.0,Australia
...,...,...,...,...,...,...
559,2018,11376070,117800.0,110645.0,7155.0,Belgium
560,2019,11431406,117103.0,108745.0,8358.0,Belgium
561,2020,11492641,113739.0,126850.0,-13111.0,Belgium
562,2021,11521238,117914.0,112291.0,5623.0,Belgium


In [ ]:
result_df.to_csv('X5_HW4_population.csv')
result_df.to_parquet('X5_HW4_population.parquet')

In [ ]:
Собранные данные лежат в этих файлах